### РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ

#### Факультет физико-математических и естественных наук  
#### Кафедра математического моделирования и искусственного интеллекта 

## ОТЧЕТ ПО ЛАБОРАТОРНОЙ РАБОТЕ № 2

#### Дисциплина: Интеллектуальный анализ данных

##### Студент:  Легиньких Галина
##### Группа:   НФИбд-02-21

##### Москва 2024
***

### Вариант № 6 
Алгоритм: FPGrowth 

День недели (поле order_dow таблицы orders): “0” 

Код департамента (поле department_id таблицы products): “8”  

Запрос: определить список товаров, которые были приобретены ровно два раза 

Показатель оценки ассоциативных правил: лифт (lift) 

## Выполнение

1. При помощи модуля sqlite3 откройте базу данных Instacart в файле instacart.db.  

In [1]:
import numpy as np
import pandas as pd
import sqlite3

# открываем базу данных
conn = sqlite3.connect('instacart.db') 

2. Загрузите таблицы departments и products в датафреймы Pandas. При помощи запроса SELECT извлеките из таблицы order_products__train записи, соответствующие указанным в индивидуальном задании дню недели (поле order_dow таблицы orders) и коду департамента (поле department_id таблицы products) и загрузите в датафрейм Pandas. Определите количество строк в полученном датафрейме, количество транзакций (покупок) и определите количество товаров (столбец product_id) в транзакциях датафрейма. 

Загрузила таблицы departments и products в датафреймы Pandas.

In [2]:
departments = pd.read_sql_query("SELECT * FROM departments", conn)
products = pd.read_sql_query("SELECT * FROM products", conn)

In [3]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [4]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


При помощи запроса SELECT извлекла из таблицы order_products__train записи, соответствующие дню недели order_dow = 0  и коду департамента department_id = 8 и загрузила в датафрейм Pandas.

In [5]:
order_products__train = pd.read_sql_query(
"""
SELECT o.*
FROM order_products__train o
JOIN orders ON o.order_id = orders.order_id
JOIN products p ON p.product_id = o.product_id
WHERE orders.order_dow = 0 AND p.department_id = 8
ORDER BY order_id DESC
""" , conn)

In [6]:
order_products__train.head()

,order_id,product_id,add_to_cart_order,reordered
0,994570,39969,6,0
1,992780,41519,2,1
2,973753,20355,1,0
3,968278,21435,2,1
4,967248,11397,5,1


Определите количество строк в полученном датафрейме. Их 888.

In [7]:
order_products__train.shape[0] # количество строк

888

Посчитала количество транзакций (покупок) и определила количество товаров (столбец product_id) в транзакциях датафрейма. 

In [8]:
len(order_products__train['order_id'].unique()) # количество транзакций

551

In [9]:
order_products__train['order_id'].value_counts() # количество товаров в транзакциях

# отсюда тоже можно было узнать количество транзакций (Length: 551)

order_id
506992     17
3396864    13
1728694    10
3341873     9
336423      8
           ..
292873      1
294722      1
296858      1
2968833     1
100000      1
Name: count, Length: 551, dtype: int64

3. Выполните к датафрейму запрос, указанный в индивидуальном задании.

Определила список товаров, которые были приобретены ровно два раза. 

In [10]:
query = """
SELECT p.product_id, p.product_name
FROM products p
JOIN order_products__train op ON p.product_id = op.product_id
GROUP BY p.product_id
HAVING COUNT(op.product_id) = 2
ORDER BY p.product_id DESC
"""
order_products_count = pd.read_sql_query(query, conn)

print("Список товаров, которые были приобретены ровно два раза:")
order_products_count

Список товаров, которые были приобретены ровно два раза:


,product_id,product_name
0,9997,Italian Style Vegetables
1,9979,Olive Oil
2,9960,Moving And Storage Tape
3,9954,Apple Nectar Juice
4,9942,Dark Japanese Style Soy Sauce
...,...,...
4905,10039,Vegetable Biryani
4906,10035,Unscented Tea Lights
4907,10026,Frozen Swiss And Bacon Petite Quiche Lorraine
4908,1001,Unseeded Plain Rye


In [11]:
order_products_count.dtypes # вот почему такая сортировка. Но переводить в числовые значения все равно нет смысла.

product_id      object
product_name    object
dtype: object

4. Постройте транзакционную базу данных из полученного датафрейма, используя в качестве идентификатора транзакции столбец order_id, а в качестве названий товаров - поле product_name из датафрейма для таблицы products, соответствующее столбцу product_id. Найдите в транзакционной базе данных транзакцию с наибольшим количеством товаров и выведите ее на экран. 

Сначала я объединила 2 таблицы и получилось 888 строк. С заданиями выше сходится (уже хорошо). Но это еще не таблица транзакций, так как каждый товар прописан отдельно.

In [12]:
# Объединяем таблицы order_products__train и products
temp = pd.merge(order_products__train, products, on="product_id")[["order_id", "product_name"]]
temp

,order_id,product_name
0,994570,Kittles Tuna & Cranberries Recipe Cat Treats
1,992780,Super Scoop Instant Clumping Unscented Cat Litter
2,973753,Proactive Health Indoor Weight & Hairball Care...
3,968278,Complete Cat Food
4,967248,Buddy Biscuits Original Oven Baked Roasted Chi...
...,...,...
883,1006312,Scented Multi-Cat Scoopable Cat Litter
884,1005229,Classic Tender Beef Feast Cat Food
885,1005229,Fancy Feast Wet Classic Turkey & Giblets Feast...
886,1000247,Whiskas® Temptations® Mix Ups Backyard Cookout...


Сгруппировала по order_id и собирала список товаров для каждой транзакции.

In [13]:
transactions = temp.groupby('order_id')['product_name'].apply(list)
transactions

order_id
100000       [Snack Sticks Chicken & Rice Recipe Dog Treats]
1000247    [Whiskas® Temptations® Mix Ups Backyard Cookou...
1005229    [Classic Tender Beef Feast Cat Food, Fancy Fea...
1006312             [Scented Multi-Cat Scoopable Cat Litter]
1011575    [Dog Snacks, Medium, Standard Size Pet Waste b...
                                 ...                        
967248     [Buddy Biscuits Original Oven Baked Roasted Ch...
968278                                   [Complete Cat Food]
973753     [Proactive Health Indoor Weight & Hairball Car...
992780     [Super Scoop Instant Clumping Unscented Cat Li...
994570        [Kittles Tuna & Cranberries Recipe Cat Treats]
Name: product_name, Length: 551, dtype: object

Нашла в транзакционной базе данных транзакцию с наибольшим количеством товаров. Это 17.

In [14]:
largest_transaction_id = transactions.apply(len).idxmax()
largest_transaction = transactions.loc[largest_transaction_id]
print(len(largest_transaction))
largest_transaction

17


['Flaked Tuna Feast Cat Food',
 'Grilled Liver & Chicken Feast in Gravy Cat Food',
 'Grilled Salmon Feast in Gravy Cat Food',
 'Classic Tender Beef Feast Cat Food',
 'Roasted Turkey Feast Cat Food',
 'Grilled Tuna Feast in Gravy Cat Food',
 'Classic Ocean Whitefish & Tuna Feast Cat Food',
 'Indoor Delights Cat Food',
 'Classic Tender Beef & Chicken Feast Cat Food',
 'Classic Tender Liver & Chicken Feast Cat Food',
 'Chunky Turkey Feast Cat Food',
 'Grilled Chicken Feast in Gravy Cat Food',
 'Chunky Chicken Feast Cat Food',
 'Classic Chopped Grill Feast Cat Food',
 'Sliced Chicken Feast in Gravy Cat Food',
 'Grilled Turkey Feast in Gravy Cat Food',
 'Tender Beef & Liver Feast Classic Wet Cat Food']

5. Постройте по транзакционной базе данных бинарную базу данных в формате датафрейма пакета mlxtend. По бинарной базе данных определите и выведите на экран три наиболее популярных товара и определите количество покупок (транзакций) этих товаров.  

Построила по транзакционной базе данных бинарную базу данных в формате датафрейма пакета mlxtend.

In [15]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
dataset_bin = te.fit(transactions).transform(transactions)
df = pd.DataFrame(dataset_bin, columns=te.columns_)
df

,174 ; Dentastix&#174 ; Large,24/7 Performance Cat Litter,24/7 Performance Clumping Litter,24/7 Performance Light Weight Cat Litter,5 Meaty Flavors Variety Dog Snacks,"6\"" Free Ranger Bully Stix",Adult Chicken & Rice Dry Dog Food,Adult Dog Food,Adult Dog Formula Dog Food,Adult Grain-Free & Poultry-Free Dog Food,...,White Meat Chicken Cutlets Natural Dog Snacks,White Meat Chicken Florentine Cat Food,White Meat Chicken Primavera Cat Food,Wild Salmon Florentine Cat Food,With Chicken & Veal in Meaty Juices Wet Dog Food,With Glade Tough Odor Solutions Cat Litter,With Ocean Fish & Salmon Cat Food,With Savory Chicken & Turkey Cat Food,Z Filets Grilled Beef Dog Treats,smart Blend Chicken & Rice Formula Dry Dog Food
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
547,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
548,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
549,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Определила и вывела на экран три наиболее популярных товара и определила количество покупок (транзакций) этих товаров.  

In [16]:
item_counts = df.sum(axis=False)
top_items = item_counts.nlargest(3)
print("Три наиболее популярных товара и количество их покупок:")
print(top_items)

Три наиболее популярных товара и количество их покупок:
Instant Action Cat Litter              23
24/7 Performance Cat Litter            18
Grain Free Chicken Formula Cat Food    17
dtype: int64


6. При помощи указанного в индивидуальном задании метода построения популярных наборов предметов постройте и выведите на экран популярный набор предметов с (абсолютной) минимальной поддержкой не менее 3, имеющий максимальную длину. При отсутствии такого набора уменьшите поддержку до 2. В случае нехватки вычислительных ресурсов (слишком долгой работы программы) при построении популярного набора предметов сокращайте число записей в наборе данных (например, делая выборку половины записей набора).  

Использовала алгоритм FPGrowth для поиска популярных наборов предметов. Задала минимальную поддержку 3 транзакции, т.е. 3 / количество транзакций.

In [17]:
from mlxtend.frequent_patterns import fpgrowth
min_support = 3 / len(df)
itemsets_fpg = fpgrowth(df, min_support=min_support, use_colnames=True)
itemsets_fpg

,support,itemsets
0,0.016334,(Snack Sticks Chicken & Rice Recipe Dog Treats)
1,0.016334,(Classic Tender Beef Feast Cat Food)
2,0.007260,(Fancy Feast Wet Classic Turkey & Giblets Feas...
3,0.007260,(Scented Multi-Cat Scoopable Cat Litter)
4,0.009074,"(Dog Snacks, Medium)"
...,...,...
116,0.005445,"(Grain Free Turkey Formula Cat Food, Natural B..."
117,0.005445,(Classic Tender Liver & Chicken Feast Cat Food...
118,0.005445,(Classic Tender Liver & Chicken Feast Cat Food...
119,0.005445,(Classic Tender Liver & Chicken Feast Cat Food...


Вывела набор, имеющий максимальную длину.

In [18]:
max_length = itemsets_fpg['itemsets'].apply(len).max()
# Фильтруем только наборы с максимальной длиной
max_length_itemsets = itemsets_fpg[itemsets_fpg['itemsets'].apply(len) == max_length]
print(f"Наборы с максимальной длиной при поддержке >= 3 транзакций:")
max_length_itemsets

Наборы с максимальной длиной при поддержке >= 3 транзакций:


,support,itemsets
115,0.005445,"(Grain Free Turkey Formula Cat Food, Grain Fre..."


7. Используя пакет mlxtend или реализацию на Python, постройте набор ассоциативных правил для полученного популярного наборов предметов с уровнем достоверности (confidence) не ниже ?.  

Я так поняла, что в конце должно быть какое-то число, но его нет. Задлала как в файле с теорией 0,7.

In [19]:
from mlxtend.frequent_patterns import association_rules

association = association_rules(itemsets_fpg, metric="confidence", min_threshold=0.7)
association

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Fancy Feast Wet Classic Turkey & Giblets Feas...,(Classic Tender Beef Feast Cat Food),0.007260,0.016334,0.005445,0.75,45.916667,0.005326,3.934664,0.985375
1,(Grilled Turkey Feast in Gravy Cat Food),(Grilled Chicken Feast in Gravy Cat Food),0.009074,0.010889,0.007260,0.80,73.466667,0.007161,4.945554,0.995421
2,"(Grain Free Turkey Formula Cat Food, Grain Fre...",(Grain Free Turkey & Salmon Formula Cat Food),0.007260,0.021779,0.005445,0.75,34.437500,0.005287,3.912886,0.978062
3,(Natural Beef & Chicken Formula Food for Cats),(Grain Free Turkey Formula Cat Food),0.007260,0.014519,0.005445,0.75,51.656250,0.005339,3.941924,0.987812
4,(Classic Ocean Whitefish & Tuna Feast Cat Food),(Classic Tender Liver & Chicken Feast Cat Food),0.007260,0.012704,0.005445,0.75,59.035714,0.005352,3.949183,0.990250
5,(Select Tender Chicken with Vegetables & Brown...,(Chunky Beef with Vegetables & Brown Rice Dog ...,0.007260,0.007260,0.005445,0.75,103.312500,0.005392,3.970962,0.997562
6,(Chunky Beef with Vegetables & Brown Rice Dog ...,(Select Tender Chicken with Vegetables & Brown...,0.007260,0.007260,0.005445,0.75,103.312500,0.005392,3.970962,0.997562


8. Для построенного набора ассоциативных правил вычислите показатель (меру) оценки ассоциативных правил, указанную в индивидуальном задании, и выведите на экран ассоциативное правило, имеющее наилучшее значение этого показателя оценки. 

У меня показатель оценки ассоциативных правил: лифт (lift). Показатель уже высчитан в прошлом задании (можно посмотреть в таблице). Вывела ассоциативное правило, имеющее наилучшее значение этого показателя оценки. Их получилось 2.

In [20]:
association[association['lift'] == association['lift']. max ()]

# аналогично комментарию в пункте 7

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
5,(Select Tender Chicken with Vegetables & Brown...,(Chunky Beef with Vegetables & Brown Rice Dog ...,0.00726,0.00726,0.005445,0.75,103.3125,0.005392,3.970962,0.997562
6,(Chunky Beef with Vegetables & Brown Rice Dog ...,(Select Tender Chicken with Vegetables & Brown...,0.00726,0.00726,0.005445,0.75,103.3125,0.005392,3.970962,0.997562
